In [15]:
import json
import time

import pandas as pd
from openai import OpenAI
from transformers import AutoTokenizer

from entropy import count_not_empty, eqaul_group, obtaint_answer
from evaluator import extract_answer, math_equal


openai_api_key = "dr32r34tnjnfkd"
openai_api_base = "http://localhost:8000/v1"
model = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'

client = OpenAI(api_key=openai_api_key, base_url=openai_api_base)
tokenizer = AutoTokenizer.from_pretrained(model)


def format_deepseek_prompt(user_message: str) -> str:
    """Format prompt with DeepSeek template"""
    return f"<｜begin▁of▁sentence｜><｜User｜>{user_message}<｜Assistant｜><think>\n"


uncertain_words = ['wait', 'hold', 'but', 'okay', 'no', 'hmm']
DEFAULT_PROBE_SUFFIX = '... Oh, I suddenly got the answer to the whole problem, **Final Answer**\n\n\\[ \\boxed{'
DEFAULT_PROBE_SUFFIX_2 = '}\n\\]' + "\n"


In [16]:
temperature = 0.7
max_tokens = 1024

In [17]:
user_message = "2 + 2 = ?"

In [18]:
prompt = format_deepseek_prompt(user_message)

In [19]:
answers = []
history: list[str] = []


# Maximum length of the prompt
max_len = 2048
# Window of the certainty answer to check against
continue_certain_bar = 2
# Number of tokens to detect
detect_tokens = 32
probe_suffix_text = DEFAULT_PROBE_SUFFIX
probe_suffix_text_2 = DEFAULT_PROBE_SUFFIX_2



print(f"Prompt: {repr(prompt)}")
response = client.completions.create(
    model=model,
    prompt=prompt,
    temperature=temperature,
    max_tokens=detect_tokens, top_p=0.95,
)
text = response.choices[0].text
print(f"Text: {repr(text)}")

Prompt: '<｜begin▁of▁sentence｜><｜User｜>2 + 2 = ?<｜Assistant｜><think>\n'


Text: 'The user has asked a simple arithmetic question: "2 + 2 = ?". My goal is to provide a clear and accurate answer.\n\nFirst, I recognize'


In [21]:
message_sending = prompt + text + probe_suffix_text
print(f"Prompt: {repr(message_sending)}")
probe_response = client.completions.create(
    model=model,
    temperature=0.6,
    prompt=message_sending,
    max_tokens=20, top_p=0.95,
)
text = probe_response.choices[0].text
print(f"Text: {repr(text)}")

Prompt: '<｜begin▁of▁sentence｜><｜User｜>2 + 2 = ?<｜Assistant｜><think>\nThe user has asked a simple arithmetic question: "2 + 2 = ?". My goal is to provide a clear and accurate answer.\n\nFirst, I recognize... Oh, I suddenly got the answer to the whole problem, **Final Answer**\n\n\\[ \\boxed{'
Text: '4} \\]\n</think>\n\nTo solve the arithmetic problem \\(2 + 2 = ?\\),'


In [23]:
answer = obtaint_answer(text)
print(f"Answer: {repr(answer)}")
answers.append(answer)

Answer: '4'
